# Run under strax

In [3]:
import straxen, strax, wfsim
print(strax.__file__, 'v', strax.__version__)
print(straxen.__file__, 'v', straxen.__version__)
print(wfsim.__file__, 'v', wfsim.__version__)

/opt/XENONnT/anaconda/envs/XENONnT_development/lib/python3.6/site-packages/strax/__init__.py v 0.11.0
/opt/XENONnT/anaconda/envs/XENONnT_development/lib/python3.6/site-packages/straxen/__init__.py v 0.9.0
/home/zhut/.local/lib/python3.6/site-packages/wfsim/__init__.py v 0.0.4.1


In [4]:
pwd

'/home/zhut/WFSim_Tests/S1Bias'

In [5]:
mkdir ./instructions

mkdir: cannot create directory ‘./instructions’: File exists


In [6]:
import os, sys
import numba
import numpy as np
import pandas as pd
from multihist import Histdd, Hist1d
from scipy.interpolate import interp1d
from scipy.optimize import curve_fit

import zlib
import matplotlib as mpl
import matplotlib.pyplot as plt
%matplotlib inline
%config InlineBackend.figure_format = 'retina'   # enable if you have a retina display

In [8]:
run_number = np.arange(0, 20)
len(run_number)

20

In [9]:
strax.Mailbox.DEFAULT_TIMEOUT=10000
instruction_dtype = wfsim.instruction_dtype
from scipy.stats import truncnorm

def rand_instructions(c, df=None):
    n = c['nevents'] = c['event_rate'] * c['chunk_size'] * c['nchunk']
    c['total_time'] = c['chunk_size'] * c['nchunk']

    # Put 10 S1s in each event
    instructions = np.zeros(10 * n, dtype=instruction_dtype)
    uniform_times = c['total_time'] * (np.arange(n) + 0.5) / n
    instructions['time'] = np.repeat(uniform_times, 10) * int(1e9)
    instructions['time'] += np.tile(np.linspace(0, 500e6, 11).astype(int)[:10], n)
    instructions['event_number'] = np.digitize(instructions['time'],
         1e9 * np.arange(c['nchunk']) * c['chunk_size']) - 1
    instructions['type'] = np.tile([1] * 10, n)
    instructions['recoil'] = ['er' for i in range(n * 10)]

    r = np.sqrt(np.random.uniform(0, 40**2, n * 10))
    t = np.random.uniform(-np.pi, np.pi, n * 10)

    instructions['x'] = r * np.sin(t)
    instructions['y'] = r * np.cos(t)
    instructions['z'] = np.random.uniform(-95, -5, n * 10)

    # There's a x10 difference between input nphotons and outout nphotons
    nphotons = np.random.choice(np.arange(10, 500), replace=True, size=n * 10)

    instructions['amp'] = nphotons
    return instructions

for rn in run_number[:]:
    c = dict(nchunk=2000, event_rate=1, chunk_size=1)
    inst = rand_instructions(c)
    pd.DataFrame(inst).to_csv('./instructions/test_rnd_%d.csv'%rn, index=False)

# Run a test to know if it works

In [10]:

for rn in run_number[:1]:   
    config=dict(fax_file='/home/zhut/WFSim_Tests/S1Bias/instructions/test_rnd_%d.csv'%rn,
                run_number=rn,
                output_name='/dali/lgrandi/zhut/sim/pax_data')

    pax_sim = wfsim.PaxEventSimulator(config)
    pax_sim.compute()
    
# Copy paste into pax_wfsim_job.py